# Discrepencies with ONNX

The notebook shows one example where the conversion leads with discrepencies if default options are used. It converts a pipeline with two steps, a scaler followed by a tree.

The bug this notebook is tracking does not always appear, it has a better chance to happen with integer features but that's not always the case. The notebook must be run again in that case.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
%matplotlib inline

## Data and first model

We take a random datasets with mostly integers.

In [3]:
import math
import numpy
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

X, y = make_regression(10000, 10)
X_train, X_test, y_train, y_test = train_test_split(X, y)

Xi_train, yi_train = X_train.copy(), y_train.copy()
Xi_test, yi_test = X_test.copy(), y_test.copy()
for i in range(X.shape[1]):
    Xi_train[:, i] = (Xi_train[:, i] * math.pi * 2 ** i).astype(numpy.int64)
    Xi_test[:, i] = (Xi_test[:, i] * math.pi * 2 ** i).astype(numpy.int64)

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

max_depth = 10

model = Pipeline([
    ('scaler', StandardScaler()),
    ('dt', DecisionTreeRegressor(max_depth=max_depth))
])

model.fit(Xi_train, yi_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('dt', DecisionTreeRegressor(max_depth=10))])

In [5]:
model.predict(Xi_test[:5])

array([-119.47819645,  186.83813161,  -84.25243139,  150.4294854 ,
        -52.375783  ])

Other models:

In [6]:
model2 = Pipeline([
    ('scaler', StandardScaler()),
    ('dt', DecisionTreeRegressor(max_depth=max_depth))
])
model3 = Pipeline([
    ('scaler', StandardScaler()),
    ('dt', DecisionTreeRegressor(max_depth=3))
])


models = [
    ('bug', Xi_test.astype(numpy.float32), model),
    ('no scaler', Xi_test.astype(numpy.float32), 
     DecisionTreeRegressor(max_depth=max_depth).fit(Xi_train, yi_train)),
    ('float', X_test.astype(numpy.float32),
     model2.fit(X_train, y_train)),
    ('max_depth=3', X_test.astype(numpy.float32),
     model3.fit(X_train, y_train))
]

## Conversion to ONNX

In [7]:
import numpy
from mlprodict.onnx_conv import to_onnx

onx = to_onnx(model, X_train[:1].astype(numpy.float32))

In [8]:
from mlprodict.onnxrt import OnnxInference

oinfpy = OnnxInference(onx, runtime="python_compiled")
print(oinfpy)

OnnxInference(...)
    def compiled_run(dict_inputs):
        # inputs
        X = dict_inputs['X']
        (variable1, ) = n0_scaler(X)
        (variable, ) = n1_treeensembleregressor(variable1)
        return {
            'variable': variable,
        }


In [9]:
import pandas

X32 = Xi_test.astype(numpy.float32)
y_skl = model.predict(X32)

obs = [dict(runtime='sklearn', diff=0)]
for runtime in ['python', 'python_compiled', 'onnxruntime1']:
    oinf = OnnxInference(onx, runtime=runtime)
    y_onx = oinf.run({'X': X32})['variable']
    delta = numpy.abs(y_skl - y_onx.ravel())
    am = delta.argmax()
    obs.append(dict(runtime=runtime, diff=delta.max()))
    obs[-1]['v[%d]' % am] = y_onx.ravel()[am]
    obs[0]['v[%d]' % am] = y_skl.ravel()[am]

pandas.DataFrame(obs)

,runtime,diff,v[1108]
0,sklearn,0.000000,129.773365
1,python,156.821991,-27.048626
2,python_compiled,156.821991,-27.048626
3,onnxruntime1,156.821991,-27.048626


The pipeline shows huge discrepencies. They appear for a pipeline *StandardScaler* + *DecisionTreeRegressor* applied in integer features. They disappear if floats are used, or if the scaler is removed. The bug also disappear if the tree is not big enough (max_depth=4 instread of 5).

In [10]:
obs = [dict(runtime='sklearn', diff=0, name='sklearn')]
for name, x32, mod in models:
    for runtime in ['python', 'python_compiled', 'onnxruntime1']:
        lonx = to_onnx(mod, x32[:1])
        loinf = OnnxInference(lonx, runtime=runtime)
        y_skl = mod.predict(X32)
        y_onx = loinf.run({'X': X32})['variable']
        delta = numpy.abs(y_skl - y_onx.ravel())
        am = delta.argmax()
        obs.append(dict(runtime=runtime, diff=delta.max(), name=name))
        obs[-1]['v[%d]' % am] = y_onx.ravel()[am]
        obs[0]['v[%d]' % am] = y_skl.ravel()[am]

df = pandas.DataFrame(obs)
df

,runtime,diff,name,v[1108],v[2237],v[158],v[1]
0,sklearn,0.000000,sklearn,129.773365,517.776400,517.776400,183.242103
1,python,156.821991,bug,-27.048626,NaN,NaN,NaN
2,python_compiled,156.821991,bug,-27.048626,NaN,NaN,NaN
3,onnxruntime1,156.821991,bug,-27.048626,NaN,NaN,NaN
4,python,0.000028,no scaler,NaN,517.776428,NaN,NaN
5,python_compiled,0.000028,no scaler,NaN,517.776428,NaN,NaN
6,onnxruntime1,0.000028,no scaler,NaN,517.776428,NaN,NaN
7,python,0.000028,float,NaN,NaN,517.776428,NaN
8,python_compiled,0.000028,float,NaN,NaN,517.776428,NaN
9,onnxruntime1,0.000028,float,NaN,NaN,517.776428,NaN


In [11]:
df.pivot("runtime", "name", "diff")

name,bug,float,max_depth=3,no scaler,sklearn
runtime,,,,,
onnxruntime1,156.821991,0.000028,0.000007,0.000028,NaN
python,156.821991,0.000028,0.000007,0.000028,NaN
python_compiled,156.821991,0.000028,0.000007,0.000028,NaN
sklearn,NaN,NaN,NaN,NaN,0.0


## Other way to converter

ONNX does not support double for TreeEnsembleRegressor but that a new operator TreeEnsembleRegressorDouble was implemented into *mlprodict*. We need to update the conversion.

In [12]:
%load_ext mlprodict

In [13]:
onx32 = to_onnx(model, X_train[:1].astype(numpy.float32))
onx64 = to_onnx(model, X_train[:1].astype(numpy.float64), 
                dtype=numpy.float64, rewrite_ops=True)
%onnxview onx64

In [14]:
X32 = Xi_test.astype(numpy.float32)
X64 = Xi_test.astype(numpy.float64)

obs = [dict(runtime='sklearn', diff=0)]
for runtime in ['python', 'python_compiled', 'onnxruntime1']:
    for name, onx, xr in [('float', onx32, X32), ('double', onx64, X64)]:
        try:
            oinf = OnnxInference(onx, runtime=runtime)
        except Exception as e:
            obs.append(dict(runtime=runtime, error=str(e), real=name))
            continue
        y_skl = model.predict(xr)
        y_onx = oinf.run({'X': xr})['variable']
        delta = numpy.abs(y_skl - y_onx.ravel())
        am = delta.argmax()
        obs.append(dict(runtime=runtime, diff=delta.max(), real=name))
        obs[-1]['v[%d]' % am] = y_onx.ravel()[am]
        obs[0]['v[%d]' % am] = y_skl.ravel()[am]

pandas.DataFrame(obs)

,runtime,diff,v[1108],v[0],real,error
0,sklearn,0.000000,129.773365,-119.478196,NaN,NaN
1,python,156.821991,-27.048626,NaN,float,NaN
2,python,0.000000,NaN,-119.478196,double,NaN
3,python_compiled,156.821991,-27.048626,NaN,float,NaN
4,python_compiled,0.000000,NaN,-119.478196,double,NaN
5,onnxruntime1,156.821991,-27.048626,NaN,float,NaN
6,onnxruntime1,NaN,NaN,NaN,double,Unable to create InferenceSession due to '[ONN...


We see that the use of double removes the discrepencies.

## OnnxPipeline

Another way to reduce the number of discrepencies is to use a pipeline which converts every steps into ONNX before training the next one. That way, every steps is either trained on the inputs, either trained on the outputs produced by ONNX. Let's see how it works.

In [15]:
from mlprodict.sklapi import OnnxPipeline

model_onx = OnnxPipeline([
    ('scaler', StandardScaler()),
    ('dt', DecisionTreeRegressor(max_depth=max_depth))
])
model_onx.fit(Xi_train, yi_train)

C:\xavierdupre\__home_\github_fork\scikit-learn\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


OnnxPipeline(steps=[('scaler',
                     OnnxTransformer(onnx_bytes=b'\x08\x06\x12\x08skl2onnx\x1a\x051.7.0"\x07ai.onnx(\x002\x00:\xf6\x01\n\xa6\x01\n\x01X\x12\x08variable\x1a\x06Scaler"\x06Scaler*=\n\x06offset=\xe0-\x90;=\x9f\xab\xad\xbc=,\xd4\x9a==\x96C\x0b\xbe=\x17H\x10\xbf=\xf7P\xb5\xbe=\xa7\x9e\x82\xc0=\'18@=gD\xb9@=G\xde\x90\xc1\xa0\x01\x06*<\n\x05scale...\xb8>=\xf9\x08.>=\xee~\xa8==\x83*&==,\x90\xa3<=-X$<=(\xc5\xa4;=\x00-";=B\xed\xa3:=#\xc3":\xa0\x01\x06:\nai.onnx.ml\x12\x1emlprodict_ONNX(StandardScaler)Z\x11\n\x01X\x12\x0c\n\n\x08\x01\x12\x06\n\x00\n\x02\x08\nb\x18\n\x08variable\x12\x0c\n\n\x08\x01\x12\x06\n\x00\n\x02\x08\nB\x0e\n\nai.onnx.ml\x10\x01')),
                    ('dt', DecisionTreeRegressor(max_depth=10))])

We see that the first steps was replaced by an object *OnnxTransformer* which wraps an ONNX file into a transformer following the *scikit-learn* API. The initial steps are still available.

In [16]:
model_onx.raw_steps_

[('scaler', StandardScaler()), ('dt', DecisionTreeRegressor(max_depth=10))]

In [17]:
models = [
    ('bug', Xi_test.astype(numpy.float32), model),
    ('OnnxPipeline', Xi_test.astype(numpy.float32), model_onx),
]

In [18]:
obs = [dict(runtime='sklearn', diff=0, name='sklearn')]
for name, x32, mod in models:
    for runtime in ['python', 'python_compiled', 'onnxruntime1']:
        lonx = to_onnx(mod, x32[:1])
        loinf = OnnxInference(lonx, runtime=runtime)
        y_skl = model_onx.predict(X32)  # model_onx is the new baseline
        y_onx = loinf.run({'X': X32})['variable']
        delta = numpy.abs(y_skl - y_onx.ravel())
        am = delta.argmax()
        obs.append(dict(runtime=runtime, diff=delta.max(), name=name))
        obs[-1]['v[%d]' % am] = y_onx.ravel()[am]
        obs[0]['v[%d]' % am] = y_skl.ravel()[am]

df = pandas.DataFrame(obs)
df

,runtime,diff,name,v[830],v[2237]
0,sklearn,0.000000,sklearn,326.571113,517.776400
1,python,322.940422,bug,649.511536,NaN
2,python_compiled,322.940422,bug,649.511536,NaN
3,onnxruntime1,322.940422,bug,649.511536,NaN
4,python,0.000028,OnnxPipeline,NaN,517.776428
5,python_compiled,0.000028,OnnxPipeline,NaN,517.776428
6,onnxruntime1,0.000028,OnnxPipeline,NaN,517.776428


Training the next steps based on ONNX outputs is better. This is not completely satisfactory... Let's check the accuracy.

In [19]:
model.score(Xi_test, yi_test), model_onx.score(Xi_test, yi_test)

(0.643341634821136, 0.6399766521329495)

Pretty close.

## Final explanation: StandardScalerFloat

We proposed two ways to have an ONNX pipeline which produces the same prediction as *scikit-learn*. Let's now replace the StandardScaler by a new one which outputs float and not double. It turns out that class *StandardScaler* computes ``X /= self.scale_`` but ONNX does ``X *= self.scale_inv_``. We need to implement this exact same operator with float32 to remove all discrepencies.

In [20]:
class StandardScalerFloat(StandardScaler):
    
    def __init__(self, with_mean=True, with_std=True):
        StandardScaler.__init__(self, with_mean=with_mean, with_std=with_std)
    
    def fit(self, X, y=None):
        StandardScaler.fit(self, X, y)
        if self.scale_ is not None:
            self.scale_inv_ = (1. / self.scale_).astype(numpy.float32)
        return self
    
    def transform(self, X):
        X = X.copy()
        if self.with_mean:
            X -= self.mean_
        if self.with_std:
            X *= self.scale_inv_
        return X

    
model_float = Pipeline([
    ('scaler', StandardScalerFloat()),
    ('dt', DecisionTreeRegressor(max_depth=max_depth))
])

model_float.fit(Xi_train.astype(numpy.float32), yi_train.astype(numpy.float32))

Pipeline(steps=[('scaler', StandardScalerFloat()),
                ('dt', DecisionTreeRegressor(max_depth=10))])

In [21]:
try:
    onx_float = to_onnx(model_float, Xi_test[:1].astype(numpy.float))
except RuntimeError as e:
    print(e)

Unable to find a shape calculator for type '<class '__main__.StandardScalerFloat'>'.
It usually means the pipeline being converted contains a
transformer or a predictor with no corresponding converter
implemented in sklearn-onnx. If the converted is implemented
in another library, you need to register
the converted so that it can be used by sklearn-onnx (function
update_registered_converter). If the model is not yet covered
by sklearn-onnx, you may raise an issue to
https://github.com/onnx/sklearn-onnx/issues
to get the converter implemented or even contribute to the
project. If the model is a custom model, a new converter must
be implemented. Examples can be found in the gallery.



We need to register a new converter so that *sklearn-onnx* knows how to convert the new scaler. We reuse the existing converters.

In [22]:
from skl2onnx import update_registered_converter
from skl2onnx.operator_converters.scaler_op import convert_sklearn_scaler
from skl2onnx.shape_calculators.scaler import calculate_sklearn_scaler_output_shapes


update_registered_converter(
    StandardScalerFloat, "SklearnStandardScalerFloat",
    calculate_sklearn_scaler_output_shapes,
    convert_sklearn_scaler)

In [23]:
models = [
    ('bug', Xi_test.astype(numpy.float32), model),
    ('FloatPipeline', Xi_test.astype(numpy.float32), model_float),
]

In [24]:
obs = [dict(runtime='sklearn', diff=0, name='sklearn')]
for name, x32, mod in models:
    for runtime in ['python', 'python_compiled', 'onnxruntime1']:
        lonx = to_onnx(mod, x32[:1])
        loinf = OnnxInference(lonx, runtime=runtime)
        y_skl = model_float.predict(X32)  # we use model_float as a baseline
        y_onx = loinf.run({'X': X32})['variable']
        delta = numpy.abs(y_skl - y_onx.ravel())
        am = delta.argmax()
        obs.append(dict(runtime=runtime, diff=delta.max(), name=name))
        obs[-1]['v[%d]' % am] = y_onx.ravel()[am]
        obs[0]['v[%d]' % am] = y_skl.ravel()[am]

df = pandas.DataFrame(obs)
df

,runtime,diff,name,v[28],v[99]
0,sklearn,0.000000,sklearn,519.540527,-464.868301
1,python,232.693756,bug,286.846771,NaN
2,python_compiled,232.693756,bug,286.846771,NaN
3,onnxruntime1,232.693756,bug,286.846771,NaN
4,python,0.000015,FloatPipeline,NaN,-464.868286
5,python_compiled,0.000015,FloatPipeline,NaN,-464.868286
6,onnxruntime1,0.000015,FloatPipeline,NaN,-464.868286


That means than the differences between ``float32(X / Y)`` and ``float32(X) * float32(1 / Y)`` are big enough to select a different path in the decision tree. ``float32(X) / float32(Y)`` and ``float32(X) * float32(1 / Y)`` are also different enough to trigger a different path. Let's illustrate that on example:

In [25]:
a1 = numpy.random.randn(100, 2) * 10
a2 = a1.copy()
a2[:, 1] *= 1000
a3 = a1.copy()
a3[:, 0] *= 1000

for i, a in enumerate([a1, a2, a3]):
    max_diff = max([
        abs(numpy.float32(x[0]) / numpy.float32(x[1]) - numpy.float32(x[0]) * numpy.float32(1/x[1]))
        for x in a])
    print(i, max_diff)

0 1.9073486e-06
1 7.450581e-09
2 0.00390625


The last random set shows very big differences, obviously big enough to trigger a different path in the graph.